In [2]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

# Задание 1

Вывести 10 самых больших по размеру треков жанра ROCK и формата MPEG (исоп

In [3]:
%%sql
SELECT *
FROM tracks as t 
JOIN genres as s ON t.GenreId = s.GenreId
WHERE s.Name like "Rock"
ORDER BY t.Bytes DESC
limit 10

 * sqlite:///chinook.db
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,GenreId_1,Name_1
1666,Dazed And Confused,137,1,1,Jimmy Page,1612329,52490554,0.99,1,Rock
620,Space Truckin',50,1,1,Blackmore/Gillan/Glover/Lord/Paice,1196094,39267613,0.99,1,Rock
1581,Dazed And Confused,127,1,1,Jimmy Page/Led Zeppelin,1116734,36052247,0.99,1,Rock
2429,We've Got To Get Together/Jingo,198,1,1,None,1070027,34618222,0.99,1,Rock
2432,Funky Piano,198,1,1,None,934791,30200730,0.99,1,Rock
621,Going Down / Highway Star,50,1,1,Gillan/Glover/Lord/Nix - Blackmore/Paice,913658,29846063,0.99,1,Rock
2427,Santana Jam,197,1,1,Carlos Santana,882834,29207100,0.99,1,Rock
2565,The Sun Road,208,1,1,"Terry Bozzio, Steve Stevens, Tony Levin",880640,29008407,0.99,1,Rock
1670,Whole Lotta Love,138,1,1,John Bonham/John Paul Jones/Robert Plant/Willie Dixon,863895,28191437,0.99,1,Rock
622,Mistreated (Alternate Version),50,1,1,Blackmore/Coverdale,854700,27775442,0.99,1,Rock


## Задание 2
Вывести названия всех групп, их песен и названия их альбомов для всех треков жанра Рок, приобретенные сотрудниками Microsoft.

In [16]:
%%sql
SELECT ar.name as "Group", t.name as "Track Name", al.title, c.Company
FROM customers as c
JOIN invoices as i ON c.CustomerId = i.CustomerId
JOIN invoice_items as it ON i.InvoiceId = it.InvoiceId
JOIN tracks as t ON it.TrackId = t.TrackId
JOIN albums as al ON t.AlbumId = al.AlbumId
JOIN artists as ar ON al.ArtistId = ar.ArtistId
JOIN genres as g ON t.GenreId = g.GenreId
WHERE (c.Company like "%Microsoft%") and (g.Name = "Rock")

 * sqlite:///chinook.db
Done.


Group,Track Name,Title,Company
Foo Fighters,No Way Back,In Your Honor [Disc 1],Microsoft Corporation
Foo Fighters,Still,In Your Honor [Disc 2],Microsoft Corporation
Foo Fighters,Razor,In Your Honor [Disc 2],Microsoft Corporation
Foo Fighters,My Hero,The Colour And The Shape,Microsoft Corporation
Guns N' Roses,Double Talkin' Jive,Use Your Illusion I,Microsoft Corporation
Guns N' Roses,The Garden,Use Your Illusion I,Microsoft Corporation
Guns N' Roses,Don't Damn Me,Use Your Illusion I,Microsoft Corporation
Guns N' Roses,Dead Horse,Use Your Illusion I,Microsoft Corporation


## Задание 3
Для каждого набора (жанр, тип медиа) вывести среднюю цену по стоимости трека и общее количество, причем вывести только те наборы, для которых все треки стоят больше 1,5$.


In [9]:
%%sql
SELECT genres.Name, media_types.Name, AVG(tracks.UnitPrice), COUNT(tracks.TrackId)
FROM tracks
JOIN media_types ON tracks.MediatypeId = media_types.MediatypeId
JOIN genres ON genres.GenreId = tracks.GenreId
GROUP BY tracks.GenreId, media_types.Name, genres.Name, tracks.MediatypeId
HAVING MIN(tracks.UnitPrice > 1.5)

 * sqlite:///chinook.db
Done.


Name,Name_1,AVG(tracks.UnitPrice),COUNT(tracks.TrackId)
Science Fiction,Protected MPEG-4 video file,1.9899999999999995,13
TV Shows,Protected MPEG-4 video file,1.990000000000001,93
Sci Fi & Fantasy,Protected MPEG-4 video file,1.9900000000000004,26
Drama,Protected MPEG-4 video file,1.9899999999999978,64
Comedy,Protected MPEG-4 video file,1.9899999999999995,17


## Задание 4
Вывести компании, сделавшие максимальное и минимальное число заказов. Необходимо исключить клиентов без компании (NULL)
Доп задание вывести след после максимального

In [72]:
%%sql

WITH f AS(
SELECT customers.company, COUNT(*) as count -- возвращает количество строк
FROM customers 
JOIN invoices ON customers.CustomerId = invoices.CustomerId
JOIN (
SELECT MAX(count_invoices) as MAXi, MIN(count_invoices) as MINi 
FROM (
SELECT customers.Company as company, COUNT(invoices.invoiceid) as count_invoices
FROM customers
JOIN invoices ON customers.CustomerId = invoices.CustomerId
WHERE customers.Company IS NOT NULL
GROUP BY company
)
)
WHERE customers.company IS NOT NULL
GROUP BY customers.company
HAVING COUNT(invoices.invoiceid) < MAXi  OR COUNT(invoices.invoiceid) > MINi
)

SELECT f.company, f.count
FROM f
WHERE f.count = (SELECT MAX(count)
                    FROM f) 
or f.count = (SELECT MIN(count)
                    FROM f)


 * sqlite:///chinook.db
Done.


company,count


## Задание 5
Для каждой компании вывести общее количестов песен, купленных по жанру поп-музыки

In [26]:
%%sql
SELECT c.company, SUM(it.quantity)
FROM customers as c
JOIN invoices as i ON c.customerid = i.customerid
JOIN invoice_items it ON i.invoiceid = it.invoiceid
JOIN tracks t ON it.trackid = t.trackid
JOIN genres g ON t.Genreid = g.genreid
WHERE g.Name = 'Pop' and c.company not like 'None'
GROUP BY c.company

 * sqlite:///chinook.db
Done.


Company,sum(it.quantity)
Embraer - Empresa Brasileira de Aeronáutica S.A.,2
JetBrains s.r.o.,4


## Задание 6
Вывести средний размер альбома в байтах.

In [31]:
%%sql
select sum(t.bytes)/(count (DISTINCT t.albumid)) as Count
from tracks as t

 * sqlite:///chinook.db
Done.


Count
338288920
